In [5]:
import requests
import glob
import os
from pathlib import Path
import re
import json
from general_purpose_agent import GeneralPurposeAgent
from parser.HDSD_excel_to_markdown import process_excel_to_markdown as hdsd_excel_extractor
from parser.docx_extractor import extract_docx_with_images as hdsd_docx_extractor
from parser.qa_docx_extractor import extract_text_and_images as qa_docx_extractor
from parser.qa_xlsx_extractor import process_excel_to_markdown as qa_excel_extractor
from parser.pdf_extractor import process_pdf_files
import pandas as pd
import pickle

ModuleNotFoundError: No module named 'parser.pdf_extractor'

In [6]:
import os
print(os.listdir('parser'))


['qa_docx_extractor.py', 'HDSD_excel_to_markdown.py', 'qa_xlsx_extractor.py', '__pycache__', 'docx_extractor.py']


In [2]:
def read_hdsd_excel(file_path):
    with open(file_path, "rb") as f:
        results = hdsd_excel_extractor(file_path, f.read())
    combined_content = []
    for result in results:
        combined_content.append(f"## Sheet {result['sheet_name']}:")
        combined_content.append(result["content"])
        combined_content.append("")  # Thêm dòng trống giữa các sheet
    
    merged_results = {
        "content": "\n".join(combined_content),
        "images": [img for result in results for img in result["images"]]
    }
    return merged_results["content"]


def read_hdsd_docx(file_path):
    return hdsd_docx_extractor(file_path)

def read_qa_docx(file_path):
    with open(file_path, "rb") as f:
        return qa_docx_extractor(file_path, f)['data']

def read_qa_excel(file_path):
    with open(file_path, "rb") as f:
        results = qa_excel_extractor(file_path, f.read())
    combined_content = []
    for result in results:
        combined_content.append(f"## Sheet {result['sheet_name']}:")
        combined_content.append(result["content"])
        combined_content.append("")  # Thêm dòng trống giữa các sheet
    
    merged_results = {
        "content": "\n".join(combined_content),
        "images": [img for result in results for img in result["images"]]
    }
    
    return merged_results['content']

def escape_json_string(raw_str: str) -> str:
    # Escape backslashes đầu tiên (\\)
    raw_str = raw_str.replace('\\', '\\\\')
    
    # Escape các ký tự đặc biệt trong JSON
    raw_str = raw_str.replace('"', '\\"')
    raw_str = raw_str.replace('\b', '\\b')
    raw_str = raw_str.replace('\f', '\\f')
    raw_str = raw_str.replace('\n', '\\n')
    raw_str = raw_str.replace('\r', '\\r')
    raw_str = raw_str.replace('\t', '\\t')

    return raw_str


def extract_json(response_text):
    pattern = r"```json\s*([\[{].*?[\]}])\s*```"
    match = re.search(pattern, response_text, re.DOTALL)
    if match:
        return json.loads(json.dumps(match.group(1)))
    return None

In [17]:
xlsx_system_prompt = """

## 🎯 Mục tiêu
Sinh các **kịch bản hội thoại nhiều lượt (multi-turn chat)** để kiểm thử khả năng hiểu và phản hồi chính xác của chatbot hỗ trợ nghiệp vụ ngân hàng, sử dụng tài liệu hướng dẫn hệ thống ACL - Loan Servicing (Secured Lending), với từng file tương ứng một loại ACL cụ thể (ví dụ: ACL Khách hàng cá nhân, ACL Khách hàng doanh nghiệp...).

---

## 📥 Đầu vào

Đầu vào là nội dung tài liệu Excel đã được chuẩn hóa theo cấu trúc sau:

```
# Sheet 0: Sheet tổng quan
[Nội dung sheet 0]

# Sheet 1
[Nội dung sheet 1]

# Sheet 2
[Nội dung sheet 2]

...
```

Trong đó:

- **Sheet 0** là tổng quan các quy trình
- **Các sheet còn lại** là hướng dẫn chi tiết từng bước hoặc quy trình cụ thể.

---

## 🧠 Nhiệm vụ của bạn

1. Dựa trên nội dung các **sheet chi tiết (từ Sheet 1 trở đi)**, tạo **một đoạn hội thoại từ 4–6 lượt** QA giữa người dùng và chatbot.

2. Mẫu kịch bản nên theo **mạch hội thoại thực tế**:

   - **Lượt 1**: Người dùng hỏi tổng quan về cách sử dụng hệ thống ACL cho loại hiện tại (ví dụ: “Cho tôi hỏi cách sử dụng hệ thống ACL Khách hàng cá nhân?”)
   - **Lượt 2**: Người dùng hỏi về một quy trình cụ thể hoặc bước cụ thể đầu tiên (ví dụ: “Tôi muốn tạo tờ trình tín dụng thì làm thế nào?”)
   - **Lượt 3**: Người dùng hỏi tiếp theo làm gì (“Sau đó tôi làm gì tiếp?” hoặc “Bước tiếp theo là gì?”)
   - **Lượt 4–6**: Tiếp tục các bước kế tiếp cho đến khi hoàn thành quy trình hoặc đến một điểm dừng hợp lý.

3. Mỗi lượt gồm:
   - `"question"`: Câu hỏi người dùng được **diễn đạt lại tự nhiên**.
   - `"answer"`: Nội dung hướng dẫn từ tài liệu, giữ đúng nguyên bản.
   - `"ref"`: Tham chiếu nguồn: `["Tên file", "Tên sheet", "Số dòng hoặc STT liên quan"]`

---

## 📄 Định dạng đầu ra

```json
{{
  "script_chat_1": {{
    "0": {{
      "question": "Câu hỏi tổng quan về ACL [loại ACL]", ## Ví dụ Hướng dẫn sử dụng ACL Khách hàng cá nhân,...
      "answer": "Câu trả lời về hướng dẫn sử dụng ACL Khách hàng cá nhân [Nội dung sheet tổng quan]",
      "ref": ["Tên file", "Tên sheet", "STT liên quan"]
    }},
    "1": {{
      "question": "Câu hỏi cụ thể bước đầu tiên",
      "answer": "Hướng dẫn chi tiết bước đầu tiên",
      "ref": ["...", "...", "..."]
    }},
    "2": {{
      "question": "Sau đó tôi làm gì?",
      "answer": "Bước kế tiếp",
      "ref": ["...", "...", "..."]
    }},
    "3": {{
      "question": "Vậy còn bước [tên bước] thì sao",
      "answer": "Nội dung sheet cụ thể của bước đấy",
      "ref": ["...", "...", "..."]    
    ...
  }}
  ...
}}
```

---

## 🔍 Lưu ý
- Luôn bắt đầu hội thoại bằng câu hỏi tổng quan về cách sử dụng hệ thống ACL của loại tương ứng với tên file.
- Nội dung câu trả lời phải đầy đủ nội dung trong sheet, không bỏ sót thông tin
- Câu hỏi phải tự nhiên, mang tính hành động thực tế.
- Tạo 4-5 kịch bản chat

---

## 📌 Ví dụ đầu vào

```
Tên file: HDSD ACL Khách hàng cá nhân.xlsx -> ACL Khách hàng cá nhân

# Sheet 0: Sheet tổng quan
1. Tờ trình tín dụng
2. Tạo khách hàng

# Sheet 1: Tờ trình tín dụng
1. Tìm kiếm khách hàng → vào menu Khách hàng > Tìm kiếm
2. Tạo tờ trình → vào menu Tín dụng > Tờ trình
3. Gửi phê duyệt

# Sheet 2: Tạo khách hàng
1. Vào menu Khách hàng > Tạo mới
2. Nhập thông tin giấy tờ, hồ sơ
```
""".strip()

In [18]:
agent = GeneralPurposeAgent(xlsx_system_prompt)

In [19]:
def process_file(file_path):
    print(f"Processing {file_path}")
    output_dir = Path(f"script_chat/{file_path.stem}")
    os.makedirs(output_dir, exist_ok=True)
    data = read_hdsd_excel(str(file_path))

    response = agent.get_response(f"Tạo FAQ giúp tôi, đây là nội dung của tài liệu HDSD: {file_path.name}:\n{data}")
    with open(output_dir / "response.pkl", "wb") as f:
        pickle.dump(response, f)
    json_response = extract_json(response["text"])
    with open(output_dir / "json_response.json", "w", encoding="utf-8") as f:
        json.dump(json.loads(json_response), f, indent=4, ensure_ascii=False)
    print(f"Done {file_path}")

# Xử lý tất cả các file .xlsx trong thư mục new_data
input_dir = Path("new_data")
# for file_path in input_dir.glob("*.xlsx"):
#     if len(list(Path(f"script_chat/{file_path.stem}").glob("*.json"))) > 0:
#         continue
#     process_file(file_path)

file_path = "new_data/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic.xlsx"
process_file(Path(file_path))

2025-06-13 14:39:24,620 - INFO - Bắt đầu xử lý nội dung file Excel
2025-06-13 14:39:24,621 - INFO - Đang tải workbook...


Processing new_data/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic.xlsx


2025-06-13 14:39:24,917 - INFO - Đã tải workbook thành công. Số sheet: 18
2025-06-13 14:39:24,919 - INFO - Đang xử lý sheet: 0
2025-06-13 14:39:24,922 - INFO - Sheet 0: 26 hàng, 9 cột có dữ liệu
2025-06-13 14:39:24,923 - INFO - Đang xử lý header của sheet 0
2025-06-13 14:39:24,927 - INFO - Đang xử lý dữ liệu của sheet 0
2025-06-13 14:39:24,929 - INFO - Đang tạo markdown cho sheet 0
2025-06-13 14:39:24,930 - INFO - Đã xử lý xong sheet 0. Số ảnh: 0
2025-06-13 14:39:24,933 - INFO - Đang xử lý sheet: 1
2025-06-13 14:39:24,934 - INFO - Sheet 1: 3 hàng, 30 cột có dữ liệu
2025-06-13 14:39:24,937 - INFO - Đang xử lý header của sheet 1
2025-06-13 14:39:24,938 - INFO - Đang xử lý dữ liệu của sheet 1
2025-06-13 14:39:24,938 - INFO - Đang tạo markdown cho sheet 1
2025-06-13 14:39:24,938 - INFO - Đã xử lý xong sheet 1. Số ảnh: 0
2025-06-13 14:39:24,939 - INFO - Đang xử lý sheet: 2
2025-06-13 14:39:24,943 - INFO - Sheet 2: 16 hàng, 3 cột có dữ liệu
2025-06-13 14:39:24,945 - INFO - Đang xử lý header 

image map: 0, dict_keys([])
image map: 2, dict_keys(['R3', 'AD3'])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 1, dict_keys(['D17'])
image map: 0, dict_keys([])
image map: 1, dict_keys(['D28'])


2025-06-13 14:39:25,200 - INFO - Đang xử lý header của sheet 7
2025-06-13 14:39:25,201 - INFO - Đang xử lý dữ liệu của sheet 7
2025-06-13 14:39:25,202 - INFO - Đang tạo markdown cho sheet 7
2025-06-13 14:39:25,202 - INFO - Đã xử lý xong sheet 7. Số ảnh: 0
2025-06-13 14:39:25,203 - INFO - Đang xử lý sheet: 8
2025-06-13 14:39:25,204 - INFO - Sheet 8: 10 hàng, 4 cột có dữ liệu
2025-06-13 14:39:25,204 - INFO - Đang xử lý header của sheet 8
2025-06-13 14:39:25,204 - INFO - Đang xử lý dữ liệu của sheet 8
2025-06-13 14:39:25,205 - INFO - Đang tạo markdown cho sheet 8
2025-06-13 14:39:25,205 - INFO - Đã xử lý xong sheet 8. Số ảnh: 0
2025-06-13 14:39:25,205 - INFO - Đang xử lý sheet: 9
2025-06-13 14:39:25,208 - INFO - Sheet 9: 16 hàng, 4 cột có dữ liệu
2025-06-13 14:39:25,211 - INFO - Đang xử lý header của sheet 9
2025-06-13 14:39:25,212 - INFO - Đang xử lý dữ liệu của sheet 9
2025-06-13 14:39:25,216 - INFO - Đang tạo markdown cho sheet 9
2025-06-13 14:39:25,218 - INFO - Đã xử lý xong sheet 9. 

image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 1, dict_keys(['D17'])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
image map: 0, dict_keys([])
Done new_data/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic.xlsx


In [20]:
import pandas as pd
import json

# Đọc file JSON
filepath = "script_chat/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic/json_response.json"
with open(filepath, "r", encoding="utf-8") as f:
    data = json.load(f)

# Tạo danh sách để lưu tất cả các câu hỏi
all_qa_pairs = []

# Duyệt qua các chat script
for chat_script_id, chat_content in data.items():
    # Duyệt qua các câu hỏi trong mỗi chat script
    for qa_id, qa_pair in chat_content.items():
        # Thêm thông tin chat_script_id và qa_id vào mỗi cặp Q&A
        qa_info = {
            "chat_script_id": chat_script_id,
            "qa_id": qa_id,
            "question": qa_pair["question"],
            "answer": qa_pair["answer"]
        }
        
        # Thêm ref nếu có
        if "ref" in qa_pair:
            qa_info["ref"] = qa_pair["ref"]
        else:
            qa_info["ref"] = ""
            
        all_qa_pairs.append(qa_info)

# Tạo DataFrame từ danh sách các cặp Q&A
df = pd.DataFrame(all_qa_pairs)

# Hiển thị 5 dòng đầu tiên của DataFrame
print(f"Tổng số câu hỏi: {len(df)}")

# Lưu DataFrame thành file Excel
output_path = "script_chat/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic/QA_detailed.xlsx"
df.to_excel(output_path, index=False)
print(f"Đã lưu DataFrame vào file: {output_path}")

Tổng số câu hỏi: 30
Đã lưu DataFrame vào file: script_chat/RM - HDSD ACL_KHCN_BSS_nga.ct_No_pic/QA_detailed.xlsx
